### CUNY Data 620 - Web Analytics, Summer 2020
**Group Homework Assignment 6**  
**Prof:** Alain Ledon  
**Members:** Misha Kollontai, Amber Ferger, Zach Alexander, Subhalaxmi Rout  

**Youtube Link:** 

### Instructions

It can be useful to be able to classify new "test" documents using already classified "training" documents. <br>A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam. <br>Here is one example of such data: http://archive.ics.uci.edu/ml/datasets/Spambase

### Import Packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import csv
from sklearn import svm
from sklearn.metrics import confusion_matrix


Using above mentioned link we will download 2 files i.e "spambase.data" and "spambase.names".<br> spambase.data file consist of features data and spam data (i.e 1 = spam, 0 = Not spam) and spambase.names file contains different features. We will extract these features from the file and apply them to the dataset to create models. 

In [2]:
# extract feature from spambase.names file.

# created an empty feature list
categories = []

# open file 
feature_file = open('spambase.names')
# remove '\n', split the text,  and match with word_freq|char_freq feature 
for line in feature_file:
    if not re.match(r'\|', line):
        line = line.rstrip()
        if re.search(r'(word_freq_|char_freq_|capital_run_length_).+', line):   
            words = line.split()
            categories.append(words[0])

categories

['word_freq_make:',
 'word_freq_address:',
 'word_freq_all:',
 'word_freq_3d:',
 'word_freq_our:',
 'word_freq_over:',
 'word_freq_remove:',
 'word_freq_internet:',
 'word_freq_order:',
 'word_freq_mail:',
 'word_freq_receive:',
 'word_freq_will:',
 'word_freq_people:',
 'word_freq_report:',
 'word_freq_addresses:',
 'word_freq_free:',
 'word_freq_business:',
 'word_freq_email:',
 'word_freq_you:',
 'word_freq_credit:',
 'word_freq_your:',
 'word_freq_font:',
 'word_freq_000:',
 'word_freq_money:',
 'word_freq_hp:',
 'word_freq_hpl:',
 'word_freq_george:',
 'word_freq_650:',
 'word_freq_lab:',
 'word_freq_labs:',
 'word_freq_telnet:',
 'word_freq_857:',
 'word_freq_data:',
 'word_freq_415:',
 'word_freq_85:',
 'word_freq_technology:',
 'word_freq_1999:',
 'word_freq_parts:',
 'word_freq_pm:',
 'word_freq_direct:',
 'word_freq_cs:',
 'word_freq_meeting:',
 'word_freq_original:',
 'word_freq_project:',
 'word_freq_re:',
 'word_freq_edu:',
 'word_freq_table:',
 'word_freq_conference:',
 '

From above all features 'spam' is not availabe, let add spam in the categories.

In [3]:
categories.append('spam')
print(categories)

['word_freq_make:', 'word_freq_address:', 'word_freq_all:', 'word_freq_3d:', 'word_freq_our:', 'word_freq_over:', 'word_freq_remove:', 'word_freq_internet:', 'word_freq_order:', 'word_freq_mail:', 'word_freq_receive:', 'word_freq_will:', 'word_freq_people:', 'word_freq_report:', 'word_freq_addresses:', 'word_freq_free:', 'word_freq_business:', 'word_freq_email:', 'word_freq_you:', 'word_freq_credit:', 'word_freq_your:', 'word_freq_font:', 'word_freq_000:', 'word_freq_money:', 'word_freq_hp:', 'word_freq_hpl:', 'word_freq_george:', 'word_freq_650:', 'word_freq_lab:', 'word_freq_labs:', 'word_freq_telnet:', 'word_freq_857:', 'word_freq_data:', 'word_freq_415:', 'word_freq_85:', 'word_freq_technology:', 'word_freq_1999:', 'word_freq_parts:', 'word_freq_pm:', 'word_freq_direct:', 'word_freq_cs:', 'word_freq_meeting:', 'word_freq_original:', 'word_freq_project:', 'word_freq_re:', 'word_freq_edu:', 'word_freq_table:', 'word_freq_conference:', 'char_freq_;:', 'char_freq_(:', 'char_freq_[:', '

'spam' feature is avaialabl ein the categories. Load data from spambase.data file. Here we will get the data and store this in csv file.

In [4]:
# get the file
data_file = open('spambase.data')
# remove extra space, split the lines and store in csv file
for file in data_file:
    space_remove = (line.strip() for line in data_file)
    lines = (line.split(",") for line in space_remove if line)
    with open('spambase.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(categories)
        writer.writerows(lines)

After generate csv file, read the csv file using read_csv().

In [5]:
data = pd.read_csv('spambase.csv', sep = ',')
print("Length of data", len(data))

Length of data 4600


In [6]:
# Count spam and non-spam
count_spam = len(data[data.spam == 1])
count_nonspam = len(data[data.spam == 0])

print("Spam: %d" %count_spam)
print("Non-spam: %d" %count_nonspam)

Spam: 1812
Non-spam: 2788


In [7]:
# print first 4 rows of data
print(data.head(4))

   word_freq_make:  word_freq_address:  word_freq_all:  word_freq_3d:  \
0             0.21                0.28            0.50            0.0   
1             0.06                0.00            0.71            0.0   
2             0.00                0.00            0.00            0.0   
3             0.00                0.00            0.00            0.0   

   word_freq_our:  word_freq_over:  word_freq_remove:  word_freq_internet:  \
0            0.14             0.28               0.21                 0.07   
1            1.23             0.19               0.19                 0.12   
2            0.63             0.00               0.31                 0.63   
3            0.63             0.00               0.31                 0.63   

   word_freq_order:  word_freq_mail:  ...  char_freq_;:  char_freq_(:  \
0              0.00             0.94  ...          0.00         0.132   
1              0.64             0.25  ...          0.01         0.143   
2              0.31     

#### Data splitting:

This table consists of 58 columns, we will devide the dataset in to 2 parts i.e the train and the test set. Train data is 75% of data and test data is 25% of data. We will create two variables input and output, input variable is from word_freq_make to  capital_run_length_total and output variable is spam.  

In [8]:
X = data.values[:, 0:57]
Y = data.values[:, 57]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

Parameter test_size is given value 0.25. It means test set will be 25% of the whole dataset and training set will be 75%. Parameter random_state is a pseudo-random number generator state used for random sampling.

#### Model 1: Support Vector machine

We will create model Support Vector machine algorithm and will create confusion matrix and the accuracy. 

In [9]:
# train model

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

svm_model = svm.SVC()
svm_model.fit(X_train, Y_train)

# Prediction
svm_pred = svm_model.predict(X_test)
svm_pred

array([0., 0., 0., ..., 0., 1., 0.])

In [10]:
# find accuracy on the Test Set

print(accuracy_score(Y_test, svm_pred))

NameError: name 'accuracy_score' is not defined

In [ ]:
# Determine Performance Metrics

print(confusion_matrix(Y_test, svm_pred))

In support vector machine(SVM) the accuracy of spam messages/emails is 81%.

**Reference: ** 